In [7]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [2]:
# Set dataset Path
with_mask_dir='/kaggle/input/face-mask-dataset/data/with_mask/'
without_mask_dir="/kaggle/input/face-mask-dataset/data/without_mask/"


In [3]:
def load_data():
    data=[]
    labels =[]
    for category in ["with_mask","without_mask"]:
        folder = os.path.join("/kaggle/input/face-mask-dataset/data",category)
        label = 0 if category == "without_mask" else 1
        for file in os.listdir(folder):
            img_path = os.path.join(folder,file)
            image = tf.keras.preprocessing.image.load_img(img_path,target_size=(128,128))
            image = tf.keras.preprocessing.image.img_to_array(image)
            image = tf.keras.applications.mobilenet.preprocess_input(image)
            data.append(image)
            labels.append(label)
            
    return np.array(data),np.array(labels)
            
        

In [4]:
X,Y = load_data()

/opt/conda/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [8]:
X.shape

(7553, 128, 128, 3)

In [9]:
Y.shape

(7553,)

In [10]:
#Splitting the datasets
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=7)

In [11]:
X_train.shape

(6042, 128, 128, 3)

In [12]:
X_test.shape

(1511, 128, 128, 3)

In [13]:
data_gen = ImageDataGenerator(
rotation_range=20,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest"
)

In [14]:
model = Sequential(
[
  Conv2D(32,(3,3),padding="same",activation="relu",input_shape=(128,128,3)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),
    
Conv2D(64,(3,3),padding="same",activation="relu"),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),
    
Flatten(),
Dense(128,activation="relu"),
    Dropout(0.4),
    Dense(1,activation="sigmoid")
    
    
])

In [15]:
model.compile(optimizer=Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])


In [16]:
history = model.fit(
data_gen.flow(X_train,y_train,batch_size=32),
validation_data=(X_test,y_test),
epochs=30)

Epoch 1/30


I0000 00:00:1729670296.746899     159 service.cc:145] XLA service 0x7ca8680045c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729670296.746970     159 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1729670296.746975     159 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  3/189 ━━━━━━━━━━━━━━━━━━━━ 14s 76ms/step - accuracy: 0.3750 - loss: 2.8592 

I0000 00:00:1729670304.048126     159 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


189/189 ━━━━━━━━━━━━━━━━━━━━ 41s 163ms/step - accuracy: 0.6689 - loss: 1.1246 - val_accuracy: 0.8438 - val_loss: 0.3660
Epoch 2/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 118ms/step - accuracy: 0.8308 - loss: 0.4020 - val_accuracy: 0.8888 - val_loss: 0.3322
Epoch 3/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 119ms/step - accuracy: 0.8427 - loss: 0.3597 - val_accuracy: 0.8518 - val_loss: 0.3285
Epoch 4/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 119ms/step - accuracy: 0.8541 - loss: 0.3477 - val_accuracy: 0.8948 - val_loss: 0.2663
Epoch 5/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 120ms/step - accuracy: 0.8642 - loss: 0.3188 - val_accuracy: 0.9100 - val_loss: 0.2321
Epoch 6/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 119ms/step - accuracy: 0.8681 - loss: 0.3074 - val_accuracy: 0.9252 - val_loss: 0.2089
Epoch 7/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 118ms/step - accuracy: 0.8831 - loss: 0.2752 - val_accuracy: 0.9279 - val_loss: 0.1830
Epoch 8/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 23s 119ms/step - accuracy: 0.8906 - loss: 0.2611 - val

In [20]:
model.save("model.h5")

In [17]:
model.evaluate(X_test,y_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9538 - loss: 0.1334


[0.13285274803638458, 0.9497022032737732]

## Creating Model with Hyperparameter tuning

In [18]:
def build_model(hp):
    model = Sequential([
        
       Conv2D(
        filters=hp.Int("conv_1_filter",min_value=20,max_value=130,step=10),
        kernel_size=hp.Choice("conv_1_kernel",values=[2,5]),
        activation="relu",
        input_shape=(128,128,3)),
        MaxPooling2D(pool_size=hp.Choice("max_pooling_1",values=[2,5])),
        Dropout(hp.Choice("dropout_1",values=[1e-1,1e-2,1e-3,1e-4])),
        
        Conv2D(
        filters=hp.Int("conv_2_filter",min_value=10,max_value=120,step=10),
        kernel_size=hp.Choice("conv_2_kernel",values=[2,5]),
        activation="relu"),
        MaxPooling2D(pool_size=hp.Choice("max_pooling_2",values=[2,5])),
        Dropout(hp.Choice("dropout_2",values=[1e-1,1e-2,1e-3,1e-4])),
        
        
        Conv2D(
        filters=hp.Int("conv_3_filter",min_value=10,max_value=120,step=10),
        kernel_size=hp.Choice("conv_3_kernel",values=[2,5]),
        activation="relu"),
        MaxPooling2D(pool_size=hp.Choice("max_pooling_3",values=[2,5])),
        Dropout(hp.Choice("dropout_3",values=[1e-1,1e-2,1e-3,1e-4])),
        
        Flatten(),
        Dense(
        units=hp.Int("dense_1",min_value=10,max_value=200,step=10),
        activation="relu"),
        Dense(1,activation="sigmoid")        
    ])
    
    model.compile(optimizer=Adam(
    hp.Choice("learning_rate",values=[1e-1,1e-2,1e-3,1e-4])),
                 loss="binary_crossentropy",
                 metrics=["accuracy"])
    return model

In [19]:
tuner = RandomSearch(
build_model,
objective="val_accuracy",
max_trials=10,
directory="tuner_weights",
project_name="face Mask Detection")

In [20]:
tuner.search(X_train,y_train,validation_data=(X_test,y_test),epochs = 4)

Trial 10 Complete [00h 00m 35s]
val_accuracy: 0.8921244144439697

Best val_accuracy So Far: 0.9523494243621826
Total elapsed time: 00h 04m 44s


In [21]:
model3 = tuner.get_best_models(num_models=1)[0]

In [22]:
model3.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 127, 127, 70)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 25, 25, 70)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25, 25, 70)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 70)     │       122,570 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 70)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 70)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 6, 6, 10)       │        17,510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 10)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 3, 10)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 90)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 120)            │        10,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 152,031 (593.87 KB)

 Trainable params: 152,031 (593.87 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model3.fit(X_train,y_train,epochs=100,batch_size=30)

Epoch 1/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - accuracy: 0.9551 - loss: 0.1200
Epoch 2/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9667 - loss: 0.0964
Epoch 3/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9743 - loss: 0.0775
Epoch 4/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9819 - loss: 0.0563
Epoch 5/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9821 - loss: 0.0529
Epoch 6/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9811 - loss: 0.0473
Epoch 7/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9896 - loss: 0.0329
Epoch 8/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9911 - loss: 0.0247
Epoch 9/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9898 - loss: 0.0291
Epoch 10/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9948 - loss: 0.0139
Epoch 11/100
202/202 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9944 - loss: 0.0156
Epoch 12/100
202/202 ━━━━━━━━

In [24]:
model3.evaluate(X_test,y_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9726 - loss: 0.2367


[0.3165361285209656, 0.9708802103996277]

In [26]:
model3.fit(X,Y)

237/237 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.9473 - loss: 0.1945


In [28]:
model3.save("model.h5")